In [0]:
file_location ="/FileStore/tables/tips.csv"
file_type="csv"
df=spark.read.csv(file_location,header=True,inferSchema=True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[5]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer=StringIndexer(inputCols=["sex","smoker","day","time"],outputCols=["sex_indexed","smoke_indexed","day_indexed","time_indexed"])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+-------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoke_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+-------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|          0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|          0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|          0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|          0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|          0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|          0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|          0.0|        1.0|         0.0|
|     26.8

In [0]:
df_r.columns

Out[11]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoke_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["tip","size","sex_indexed","smoke_indexed","day_indexed","time_indexed"],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [0]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
finalize_data=output.select("Independent Features","total_bill")

In [0]:
finalize_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalize_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features',labelCol='total_bill')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[22]: DenseVector([2.654, 3.7572, -1.3045, 2.4025, -0.2297, -1.1251])

In [0]:
regressor.intercept

Out[23]: 2.2783983683722435

In [0]:
pred_result=regressor.evaluate(test_data)

In [0]:
pred_result.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.51|13.110397588273004|
| (6,[0,1],[2.0,2.0])|     13.37|15.100920274812138|
|(6,[0,1],[2.72,2.0])|     13.28|17.011822053889706|
| (6,[0,1],[3.0,2.0])|      14.0|17.754950523530983|
|(6,[0,1],[3.15,3.0])|     20.08| 21.91028576533991|
|(6,[0,1],[3.35,3.0])|     20.65| 22.44109181508368|
|(6,[0,1],[3.39,2.0])|     11.61| 18.79002232053133|
| (6,[0,1],[4.3,2.0])|      21.7| 21.20518984686548|
| (6,[0,1],[5.0,3.0])|     31.27| 26.82024172546977|
|(6,[0,1],[6.73,4.0])|     48.27|35.168944760254476|
|[1.0,1.0,1.0,1.0,...|      3.07| 9.787683240578232|
|[1.01,2.0,1.0,0.0...|     16.99|10.939245088144563|
|[1.1,2.0,1.0,1.0,...|      12.9| 13.81031696995122|
|[1.32,2.0,0.0,0.0...|      9.68|13.066490598607425|
|[1.44,2.0,0.0,1.0...|      7.74|16.017183387875647|
|[1.48,2.0,0.0,0.0...|      8.52|12.1363854757

In [0]:
regressor.predict

Out[27]: <bound method JavaPredictionModel.predict of LinearRegressionModel: uid=LinearRegression_8b0c96554f64, numFeatures=6>

In [0]:
pred_result.r2

Out[28]: 0.70961947262968

In [0]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError,pred_result.rootMeanSquaredError

Out[29]: (4.237356001012154, 26.99478983354572, 5.195651050017285)